In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("lab-ml").getOrCreate()
spark

In [3]:

df = spark.read.parquet("s3://aws-logs-780494741284-us-east-1/dataset/project_graph_data")

In [4]:
df.createOrReplaceTempView("df")
df.printSchema()

root
 |-- source: string (nullable = true)
 |-- target: string (nullable = true)



In [5]:
df.take(5)

[Row(source='http//0.ssbet138.cn/WWW.XPJ80886.COM/', target='/WWW.HB6611.COM/'),
 Row(source='http//0.ssbet138.cn/WWW.XPJ80886.COM/', target='/WWW.SX1822.COM/'),
 Row(source='http//0.ssbet138.cn/WWW.XPJ80886.COM/', target='/WWW.DK8883.COM/'),
 Row(source='http//0.ssbet138.cn/WWW.XPJ80886.COM/', target='/WWW.287720.COM/'),
 Row(source='http//0.ssbet138.cn/WWW.XPJ80886.COM/', target='/WWW.MG8541.COM/')]

In [7]:
df.take(500)

[Row(source='http//0.ssbet138.cn/WWW.XPJ80886.COM/', target='/WWW.HB6611.COM/'),
 Row(source='http//0.ssbet138.cn/WWW.XPJ80886.COM/', target='/WWW.SX1822.COM/'),
 Row(source='http//0.ssbet138.cn/WWW.XPJ80886.COM/', target='/WWW.DK8883.COM/'),
 Row(source='http//0.ssbet138.cn/WWW.XPJ80886.COM/', target='/WWW.287720.COM/'),
 Row(source='http//0.ssbet138.cn/WWW.XPJ80886.COM/', target='/WWW.MG8541.COM/'),
 Row(source='http//0.ssbet138.cn/WWW.XPJ80886.COM/', target='/WWW.1382121.COM/'),
 Row(source='http//0.ssbet138.cn/WWW.XPJ80886.COM/', target='/WWW.3615005.COM/'),
 Row(source='http//0.ssbet138.cn/WWW.XPJ80886.COM/', target='/WWW.7760.NET/'),
 Row(source='http//0.ssbet138.cn/WWW.XPJ80886.COM/', target='/WWW.HG123777.COM/'),
 Row(source='http//0.ssbet138.cn/WWW.XPJ80886.COM/', target='/WWW.CP85555.COM/'),
 Row(source='http//0.ssbet138.cn/WWW.XPJ80886.COM/', target='/WWW.99662.COM/'),
 Row(source='http//0.ssbet138.cn/WWW.XPJ80886.COM/', target='/WWW.13300007.COM/'),
 Row(source='http//0.ssb